In [1]:
# Import necessary libraries
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models import resnet18, ResNet18_Weights
import torch.nn as nn
import torch
from tqdm import tqdm  # For displaying progress bars during training/evaluation

# Custom dataset for weather classification
class WeatherClassifierDataset(Dataset):
    def __init__(self, root_dir, transform=None, train=True):
        self.samples = []  # List to hold (image_path, label) tuples
        self.transform = transform
        mode = 'train' if train else 'test'  # Choose folder based on train/test mode

        # Loop through the two classes: rain and haze
        for label, (weather_type, subfolder) in enumerate([('rain', 'rainy'), ('haze', 'hazy')]):
            folder = os.path.join(root_dir, weather_type, mode, subfolder)  # Construct full folder path
            for img in os.listdir(folder):  # Iterate over all image files
                self.samples.append((os.path.join(folder, img), label))  # Append full path and label

    def __len__(self):
        return len(self.samples)  # Return total number of samples

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]  # Get image path and label at index
        image = Image.open(img_path).convert('RGB')  # Load image and convert to RGB
        if self.transform:
            image = self.transform(image)  # Apply transformations if any
        return image, label  # Return transformed image and label

# Define root directory for the dataset
root_dir = "../Data"

# Define transformations for the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 (suitable for ResNet)
    transforms.ToTensor()  # Convert PIL image to tensor
])

# Create dataset instances for training and testing
train_ds = WeatherClassifierDataset(root_dir, transform, train=True)
test_ds = WeatherClassifierDataset(root_dir, transform, train=False)

# Wrap datasets in DataLoader for batching and shuffling
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)  # Shuffle during training
test_loader = DataLoader(test_ds, batch_size=16, shuffle=False)   # No shuffle during testing


In [8]:
# Model Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, else fallback to CPU
print("Using device:", device, torch.cuda.get_device_name())  # Print selected device

# Load pre-trained ResNet18 model with ImageNet weights
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

# Replace the final fully connected layer to match our binary classification task (rain vs haze)
model.fc = nn.Linear(model.fc.in_features, 2)

# Load the model weights from a previously saved checkpoint (epoch 5)
model.load_state_dict(torch.load("model_classifier_epoch5.pth"))

# Move model to the selected device (GPU or CPU)
model.to(device)

# Define the loss function (CrossEntropy for multi-class classification)
criterion = nn.CrossEntropyLoss()

# Use Adam optimizer with a learning rate of 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop from epoch 5 to 9 (inclusive of 5, exclusive of 10)
for epoch in range(5, 10):
    model.train()  # Set model to training mode
    total_loss = 0  # To accumulate loss over the epoch

    # tqdm progress bar for training batches
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}", unit="batch")

    # Iterate over training data
    for imgs, labels in pbar:
        imgs, labels = imgs.to(device), labels.to(device)  # Move data to device

        outputs = model(imgs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss

        optimizer.zero_grad()  # Clear previous gradients
        loss.backward()        # Backpropagation
        optimizer.step()       # Update model parameters

        total_loss += loss.item()  # Accumulate loss

        # Update progress bar with current batch loss
        pbar.set_postfix(loss=loss.item())

    # Print average loss for the epoch
    print(f"Epoch {epoch+1} completed. Avg Loss: {total_loss / len(train_loader):.4f}")
    
    # Save model state after each epoch for checkpointing
    torch.save(model.state_dict(), f"model_classifier_epoch{epoch+1}.pth")


Using device: cuda NVIDIA GeForce RTX 3050 Ti Laptop GPU


Epoch 6: 100%|███████████████████████████████████████████████████████| 1232/1232 [03:59<00:00,  5.15batch/s, loss=9e-5]


Epoch 6 completed. Avg Loss: 0.0069


Epoch 7: 100%|████████████████████████████████████████████████████| 1232/1232 [03:20<00:00,  6.15batch/s, loss=3.48e-5]


Epoch 7 completed. Avg Loss: 0.0042


Epoch 8:   2%|█                                                      | 24/1232 [00:04<03:37,  5.55batch/s, loss=0.0026]


KeyboardInterrupt: 

In [9]:
# Import evaluation metrics
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm  # For progress bar during evaluation

# Reload the test dataset (ensuring consistency if test dataset was changed during training)
test_dataset = WeatherClassifierDataset(root_dir, transform=transform, train=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)  # No shuffling for evaluation

# Select device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device, torch.cuda.get_device_name(device.index if device.index is not None else 0))

# Load the model and modify the final layer for binary classification
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, 2)

# Load model weights from epoch 7 checkpoint
model.load_state_dict(torch.load("model_classifier_epoch7.pth"))
model.to(device)  # Move model to selected device
model.eval()      # Set model to evaluation mode (disables dropout, etc.)

# Initialize lists to store all predictions and true labels
all_preds, all_labels = [], []

# Disable gradient calculation for inference (saves memory and improves speed)
with torch.no_grad():
    for imgs, labels in tqdm(test_loader, desc="Evaluating", unit="batch"):
        imgs = imgs.to(device)  # Move images to device
        outputs = model(imgs)   # Forward pass
        preds = torch.argmax(outputs, dim=1).cpu().numpy()  # Get predicted class indices
        all_preds.extend(preds)  # Store predictions
        all_labels.extend(labels.numpy())  # Store true labels

# Print overall test accuracy
print("\nClassifier Test Accuracy:", accuracy_score(all_labels, all_preds))

# Print detailed classification report (precision, recall, F1-score)
print(classification_report(all_labels, all_preds, target_names=["Rain", "Haze"]))


Using device: cuda NVIDIA GeForce RTX 3050 Ti Laptop GPU


Evaluating: 100%|█████████████████████████████████████████████████████████████████| 238/238 [00:28<00:00,  8.34batch/s]



Classifier Test Accuracy: 0.9889473684210527
              precision    recall  f1-score   support

        Rain       1.00      0.99      0.99      2800
        Haze       0.96      1.00      0.98      1000

    accuracy                           0.99      3800
   macro avg       0.98      0.99      0.99      3800
weighted avg       0.99      0.99      0.99      3800

